# Cropping a domain using the lat/lon convex hull of another domain

This notebook demonstrates how convex hull based cropping is implemented in `mllam-data-prep`. To actually use this feature see [the details in the README](../README.md#cropping-a-domain-using-the-latlon-convex-hull-of-another-domain).

This method crops a domain using the lat/lon convex hull of another domain. This is useful when a) you have two datasets on two different overlapping domains where you only want to keep the overlapping part, or b) in the case where you want to run limited-area simulations and need to create a dataset that provides the boundary conditions for the limited-area domain.

In [ ]:
import tests.data as testdata
import tempfile

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import mllam_data_prep as mdp
import tests.data as testdata
from mllam_data_prep.ops import cropping    

We will start by creating the some synthetic data on two different domains where one sits within the other, mimicking the scenario where you have a high-resolution dataset within a coarser dataset.

In [ ]:
tmpdir = tempfile.TemporaryDirectory()
l = 500 * 1.0e3  # length and width of domain in meters
N = 50  # number of grid points in each direction
config_lam = testdata.create_input_datasets_and_config(
    identifier="lam",
    data_categories=["state"],
    tmpdir=tmpdir,
    xlim=[-l/2.0, l/2.0],
    ylim=[-l/2.0, l/2.0],
    nx=N, ny=N,
    add_latlon=True

)
# make the global domain twice as large as the LAM domain so that the lam
# domain is contained within the global domain, but half the number of grid
# points in each direction
config_global = testdata.create_input_datasets_and_config(
    identifier="global",
    data_categories=["state"],
    tmpdir=tmpdir,
    xlim=[-l, l],
    ylim=[-l, l],
    nx=N//2, ny=N//2,
    add_latlon=True
)


In [ ]:
ds_lam = mdp.create_dataset(config=config_lam)
ds_global = mdp.create_dataset(config=config_global)

In [ ]:
ds_lam

In [ ]:
ds_global

Let's first make a mask that indicates which of the coarser domain grid-points are within the convex hull of the high-resolution grid points coordinates

In [ ]:
da_interior_mask = cropping.create_convex_hull_mask(ds=ds_global, ds_reference=ds_lam)

In [ ]:
da_interior_mask = da_interior_mask.where(da_interior_mask, drop=True)

In [ ]:
ds_reference = ds_lam
ds = ds_global

fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
xr.plot.scatter(ax=ax, ds=ds, x="lon", y="lat", label="ds", transform=ccrs.PlateCarree())
xr.plot.scatter(ax=ax, ds=ds_reference, x="lon", y="lat", label="ds_ref", transform=ccrs.PlateCarree())
xr.plot.scatter(ax=ax, c=da_interior_mask, ds=da_interior_mask.to_dataset(name="mask"), x="lon", y="lat", marker="x", label="ds (interior)")
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.legend()
plt.show()
    

Next we will for the points from the larger domain and exterior to this convex hull (i.e. points excluded by the mask created above) compute the distance to the nearest point of the inner domain.

In [ ]:
da_dist = cropping.distance_to_convex_hull_boundary(ds=ds_global, ds_reference=ds_lam)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()), figsize=(10, 4))
xr.plot.scatter(ax=ax, ds=ds_reference, x="lon", y="lat", label="ds_ref", marker=".", transform=ccrs.PlateCarree())
xr.plot.scatter(ax=ax, hue="dist", ds=da_dist.to_dataset(name="dist"), x="lon", y="lat", marker="x", label="ds (distance)", add_colorbar=True)
ax.coastlines()
ax.gridlines(draw_labels=["top", "left"])

And finally we will use this distance to create a plot that includes a margin around the convex hull of the inner domain.

In [ ]:
max_dist = 1.5 # in degrees
da_global_margin_crop = cropping.crop_with_convex_hull(
    ds=ds_global, ds_reference=ds_lam, margin_thickness=max_dist
)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()), figsize=(10, 4))
xr.plot.scatter(ax=ax, ds=ds_reference, x="lon", y="lat", label="ds_ref", marker=".", transform=ccrs.PlateCarree())
xr.plot.scatter(ax=ax, ds=da_global_margin_crop, x="lon", y="lat", marker="x", label=f"ds (margin crop, {max_dist}deg)", transform=ccrs.PlateCarree())
ax.coastlines()
ax.legend()
ax.gridlines(draw_labels=["top", "left"])

In [ ]:
max_dist = 1.5 # in degrees
ds_global_margin_crop_without_interior = cropping.crop_with_convex_hull(
    ds=ds_global, ds_reference=ds_lam, margin_thickness=max_dist, include_interior_points=False
)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()), figsize=(10, 4))
xr.plot.scatter(ax=ax, ds=ds_reference, x="lon", y="lat", label="ds_ref", marker=".", transform=ccrs.PlateCarree())
xr.plot.scatter(ax=ax, ds=ds_global_margin_crop_without_interior, x="lon", y="lat", marker="x", label=f"ds (margin crop, {max_dist}deg)", transform=ccrs.PlateCarree())
ax.coastlines()
ax.legend()
ax.gridlines(draw_labels=["top", "left"])